# RFM ANALYSIS

In [ ]:
# RFM Analysis (Recency, Frequency, Monetary Value Analysis) is used for customer profiling and segmentation based on purchasing behavior.
# These metrics provide information about customer engagement, loyalty and value to business

# Recency (date when customer made last purchase)
# Frequency (how often customer made purchase)
# Monetary Value (Total amount spent on purchase)

# RFM analysis scores are given in range of 1 to 5.
# Recency [5,4,3,2,1] - Higher Score for Lower Recency
# Frequency[1,2,3,4,5] - Higher for Score for Higher Frequency
# Monetary Value [1,2,3,4,5] - Higher Score for Higher Monetary Value

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [18]:
df = pd.read_csv(r"C:\Users\zaida\OneDrive\Desktop\Machine Learning\rfm_data.csv")

In [19]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [21]:
# Convert date to datetime format

In [22]:
df["PurchaseDate"] = pd.to_datetime(df.PurchaseDate)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [24]:
from datetime import datetime

### Calculating Recency

In [25]:
# To calculate recency we have to substract purchase date from current date with the help of this we can 
# get the time when customer last purchase.

In [26]:
df["Recency"] = (datetime.now().date()-df["PurchaseDate"].dt.date)/np.timedelta64(1,'D')

# df['Recency']: This part of the code assigns a new column called 'Recency' to the DataFrame df.
# This column will store the recency values calculated based on the difference between the current date and the purchase date.

# (datetime.now().date() - df.PurchaseDate.dt.date): This calculates the time difference between the
# current date and the purchase date for each record in the DataFrame df.

# np.timedelta64(1, 'D'): This part converts the time difference into the number of days.

### Calculating Frequency

In [29]:
df_freq = df.groupby("CustomerID")["OrderID"].count().reset_index()

# df.groupby('CustomerID'): This groups the DataFrame df by the 'CustomerID' column.
# This means that the data will be split into groups based on unique customer IDs.

# ['OrderID'].count(): Within each group, this part counts the number of occurrences of the 'OrderID' column. Essentially, 
# it calculates the number of orders made by each customer.

# .reset_index(): This resets the index of the resulting DataFrame. By default, the groupby operation creates
# a multi-level index. Using reset_index()converts the grouped results into a DataFrame with a default integer index.

In [30]:
# Renaming OrderId to Frequency
df_freq.rename(columns={"OrderID":"Frequency"},inplace=True)

In [34]:
# Adding df_freq to df 
df = df.merge(df_freq,on="CustomerID",how="left")

### Calculate Monetary

In [36]:
df_monetary = df.groupby("CustomerID")["TransactionAmount"].sum().reset_index()
# df.groupby("CustomerID"): This groups the DataFrame df by the 'CustomerID' column. 
# This means that the data will be split into groups based on unique customer IDs.

# ["TransactionAmount"].sum(): Within each group, this part calculates the sum of the 'TransactionAmount' column. 
# It calculates the total amount spent by each customer across all transactions.

In [37]:
df_monetary.rename(columns={"TransactionAmount":"MonetaryValue"},inplace=True)

In [38]:
df = df.merge(df_monetary,on="CustomerID",how="left")

In [40]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,304.0,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,304.0,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,304.0,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,304.0,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,304.0,1,739.56


In [41]:
# Define Scales or Score for RFM 
recency_score=[5,4,3,2,1]
frequency_score=[1,2,3,4,5]
monetary_score=[1,2,3,4,5]

In [45]:
# Calculate Recency_score

df["Recency_Score"] = pd.cut(df.Recency,bins=5,labels=recency_score)

# bins=5: This parameter specifies the number of bins or intervals to divide the data into. In this case,
# the recency values will be divided into 5 bins.

In [46]:
# Calculate frequency_score

df["Frequency_Score"] = pd.cut(df.Frequency,bins=5,labels=frequency_score)

In [47]:
# Calculate Monetray Score

df['Monetary_Score'] = pd.cut(df.Monetary_Score,bins=5,labels=monetary_score)

In [48]:
# Change data type
df.Recency_Score = df.Recency_Score.astype(int)
df.Monetary_Score = df.Monetary_Score.astype(int)
df.Frequency_Score = df.Frequency_Score.astype(int)

In [49]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency_Score,Frequency_Score,Monetary_Score
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,304.0,1,943.31,1,1,2
1,2188,2023-04-11,463.70,Product A,176819,London,304.0,1,463.70,1,1,1
2,4608,2023-04-11,80.28,Product A,340062,New York,304.0,1,80.28,1,1,1
3,2559,2023-04-11,221.29,Product A,239145,London,304.0,1,221.29,1,1,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,304.0,1,739.56,1,1,2


In [50]:
df["RFM_Score"] = df["Recency_Score"]+df["Frequency_Score"]+df["Monetary_Score"]

In [51]:
# Create a new variable Customer_Segment

In [52]:
Segment_Labels = ["Low_Value","Medium_Value","High_Value"]

In [53]:
df["Customer_Segment"] = pd.qcut(df.RFM_Score, q=3, labels=Segment_Labels)

# This part of the code segments the RFM scores into quantiles using the pd.qcut() function:
# q=3: This parameter specifies the number of quantiles to divide the data into. 
# In this case, the RFM scores will be divided into 3 quantiles.

In [54]:
df.Customer_Segment.value_counts()

Customer_Segment
Low_Value       435
Medium_Value    386
High_Value      179
Name: count, dtype: int64

In [55]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,Customer_Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,304.0,1,943.31,1,1,2,4,Low_Value
1,2188,2023-04-11,463.70,Product A,176819,London,304.0,1,463.70,1,1,1,3,Low_Value
2,4608,2023-04-11,80.28,Product A,340062,New York,304.0,1,80.28,1,1,1,3,Low_Value
3,2559,2023-04-11,221.29,Product A,239145,London,304.0,1,221.29,1,1,1,3,Low_Value
4,9482,2023-04-11,739.56,Product A,194545,Paris,304.0,1,739.56,1,1,2,4,Low_Value


In [56]:
rfm_segments=[]
for row in df.RFM_Score:
    if row>=9:
        rfm_segments.append("Champions")
    elif row>=6:
        rfm_segments.append("Loyalist")
    elif row>=5:
        rfm_segments.append("At Risk")
    elif row>=4:
        rfm_segments.append("Can't Lose")
    else:
        rfm_segments.append("Lost")
df['RFM_Segment']=rfm_segments

In [57]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,Customer_Segment,RFM_Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,304.0,1,943.31,1,1,2,4,Low_Value,Can't Lose
1,2188,2023-04-11,463.70,Product A,176819,London,304.0,1,463.70,1,1,1,3,Low_Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,304.0,1,80.28,1,1,1,3,Low_Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,304.0,1,221.29,1,1,1,3,Low_Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,304.0,1,739.56,1,1,2,4,Low_Value,Can't Lose


In [58]:
df.RFM_Segment.value_counts()

RFM_Segment
Loyalist      503
At Risk       180
Can't Lose    173
Lost           82
Champions      62
Name: count, dtype: int64